In [1]:
import os
import sys
import argparse
import pandas as pd
import time
from datetime import datetime
import matplotlib as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, SGD

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics as metrics

from keras.models import load_model
from keras.models import model_from_json

import tensorflow as tf
from tensorflow.python.client import device_lib

import cv2
import json
from fastai_utils import *
from imgaug import augmenters as iaa
import imgaug as ia
from model_utils import *

start = time.time()

os.chdir('/data/data_backup_affine/Data_v2')

Using TensorFlow backend.


In [12]:
bs = 64
nb_epoch = 2
train_data_path = "./train"
validation_data_path = "./val"
test_path = "./final_test"
annotation_path = "./final_data_annotation_with_col8.csv"
model_weights_path = "./models/model_weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
img_channels = 3
img_rows = 224
img_cols = 224

In [13]:
## Assign GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [14]:
model_split_1 = 'res4a_branch2a'
model_split_2 = 'fc_start'

## Defining model name
date = datetime.now().strftime("%m%d%Y")
model_name = 'resnet50_regression'
save_model_path = './models/iterations/' + date + '_' + model_name + '/'
if not os.path.exists(save_model_path):
    os.mkdir(save_model_path)

In [15]:
## Reading Data Annotation
data = pd.read_csv(annotation_path, encoding = 'latin-1')
data.head()

,Unnamed: 0,database,key,accident_no,img_name,comment,Col2,Col3,Col4,Col5,Col6,Col7,Col8,filename,accident_code,split
0,1,Data_DBCar_Cases,130010,130010DV X218 frontal gegen Leitplanke,AN_03,NaN,1,F,R,E,E,2,60.0,130010DV X218 frontal gegen Leitplanke_AN_03,130010DV X218 frontal gegen Leitplanke_01FREE2,train
1,2,Data_DBCar_Cases,130010,130010DV X218 frontal gegen Leitplanke,AN_15,NaN,1,F,R,E,E,2,60.0,130010DV X218 frontal gegen Leitplanke_AN_15,130010DV X218 frontal gegen Leitplanke_01FREE2,train
2,3,Data_DBCar_Cases,130010,130010DV X218 frontal gegen Leitplanke,AN_16,NaN,1,F,R,E,E,2,60.0,130010DV X218 frontal gegen Leitplanke_AN_16,130010DV X218 frontal gegen Leitplanke_01FREE2,train
3,4,Data_DBCar_Cases,130010,130010DV X218 frontal gegen Leitplanke,AN_17,NaN,1,F,R,E,E,2,60.0,130010DV X218 frontal gegen Leitplanke_AN_17,130010DV X218 frontal gegen Leitplanke_01FREE2,train
4,5,Data_DBCar_Cases,130032,130032DV C204 Frontalkollision schleudernd geg...,DA_01,NaN,1,F,D,E,W,3,65.0,130032DV C204 Frontalkollision schleudernd geg...,130032DV C204 Frontalkollision schleudernd geg...,train


In [16]:
unq_acc = data[['accident_no','accident_code']].drop_duplicates()
accident_name = unq_acc.loc[unq_acc['accident_no'].duplicated(),'accident_no']
print(list(accident_name))

['180060DV X253 frontal gegen Bahnbetriebshäuschen', '150060DV S212 Polizei Seitenkollision durch Passat und frontal gegen Mast', '140052DV C117 Seitenkollision durch Vito und seitlich gegen Mast']


In [17]:
data = data.loc[~data['accident_no'].isin(accident_name),:]
print('after removing dupicate accidents ... \n', data.shape)
data = data.loc[~pd.isnull(data['Col8']),:]
print('after removing nulls in col8 .. \n', data.shape)

after removing dupicate accidents ... 
 (557, 16)
after removing nulls in col8 .. 
 (499, 16)


In [18]:
## Reading Data Annotation
data['filename'] = data['filename'].apply(lambda x: x+'.jpg')
columns = 'Col8'
    
## Preparing dataframe that will be used as an input to flow_from_directory
image_files = pd.DataFrame({'filename': data['filename'], 'targets':data[columns]})
image_files.shape

(499, 2)

In [19]:
## Defining augmentation techniques
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        # iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.OneOf([
            sometimes(iaa.CropAndPad(percent=(-0.05, 0.1))), # zoom in
            sometimes(iaa.Affine(scale={"x": (0.6, 1.2), "y": (0.6, 1.2)})) # zoom out
        ]), 
        # execute 1 to 2 of the following augmenters per image
        sometimes(iaa.Affine(translate_px={"x": (0, 25), "y": (0, 25)}, # horizontal/vertical shift
                            rotate = (-25, 25),
                            shear = (-15, 15))), 
        iaa.SomeOf((1, 4),
                   [iaa.OneOf([
                       iaa.PerspectiveTransform(scale=(0.01, 0.07)),
                       iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)),
                       iaa.PiecewiseAffine((0.0, 0.01)), # local distortions
                       iaa.GaussianBlur(sigma=(0, 0.7))
                   ]),
                    sometimes(iaa.Dropout((0.01, 0.02), per_channel=0.5)),
                    iaa.AdditiveGaussianNoise(loc=32, scale=0.01*255), # white noise
                    iaa.Add((-20, 50)), # brightness
                    iaa.OneOf([
                        iaa.LinearContrast(alpha=(0.5,1.2), per_channel = True),
                        iaa.ContrastNormalization((0.5, 1.0))
                    ])
            ], random_order=True)
    ], random_order=True)

In [20]:
## Creating image generators for train and validation
img_train_gen = ImageDataGenerator(rescale=1/255, preprocessing_function = seq.augment_image)
img_val_gen = ImageDataGenerator(rescale=1/255)

## Splitting annotated images into training and validation
## train
train_ids = list(set(data.loc[data['split'] == 'train', 'filename']))
train_df = image_files.loc[image_files['filename'].isin(train_ids),:].drop_duplicates().reset_index(drop = True)
print(str(len(train_ids)) + ' Training Images')
train_gen = img_train_gen.flow_from_dataframe(train_df, train_data_path,
                                  x_col = 'filename', y_col = 'targets',
                                  has_ext = True, target_size = (img_rows, img_cols),
                                  color_mode = 'rgb', batch_size = bs, class_mode = 'other')

## validation
val_ids = list(set(data.loc[data['split'] == 'val', 'filename']))
val_df = image_files.loc[image_files['filename'].isin(val_ids),:].drop_duplicates().reset_index(drop = True)
print(str(len(val_ids)) + ' Validation Images')
val_gen = img_train_gen.flow_from_dataframe(val_df, validation_data_path,
                                  x_col = 'filename', y_col = 'targets',
                                  has_ext = True, target_size = (img_rows, img_cols),
                                  color_mode = 'rgb', batch_size = bs, class_mode = 'other')

431 Training Images
Found 431 images.
68 Validation Images
Found 68 images.


In [21]:
def build_model(base_model, img_channels, img_w, img_h):    
    """Builds and returns a learning model.
    img_channels: the number of channels in the input images (1 for grayscale,
        or 3 for RGB).
    img_w: the width (in pixels) of the input images.
    img_h: the height of the input images.
    classes_list: the number of classes that the data will have (for each code) - this dictates
        the number of values produced in the output layer.
    Returns:
    A deep neural network model.
    """
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu', name = 'fc_start')(x)
    x = Dropout(0.4)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    out = Dense(1, activation='linear')(x)

    return out

In [22]:
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

## Defining the input image size
SHAPE = (img_rows, img_cols, img_channels)

base_model = ResNet50(include_top=False, weights=None, input_shape = SHAPE)
base_model.load_weights(model_weights_path)

# print('freezing entire base model ...\n')
# # Freezing entire base model
# for layer in base_model.layers[:]:
#     layer.trainable = False

model_output = build_model(base_model, img_channels, img_rows, img_cols)
model = Model(inputs = base_model.input, outputs = model_output)

model_checkpoint = ModelCheckpoint(save_model_path + model_name +'_checkpoint.h5',
                                   monitor='val_loss', 
                                   mode = 'auto', save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor='val_loss', mode = 'auto',patience = 5, verbose=2)
# For cyclical Learning Rate
sched = LR_Cycle(iterations = np.ceil(len(train_ids)/bs),
                 cycle_mult = 2)

cbks = [model_checkpoint,early_stopping,sched]
# cbks1 = [model_checkpoint,early_stopping]
# cbks2 = [model_checkpoint,early_stopping, sched]

# opt = SGD(lr = 5 * 1e-2, momentum = 0.9)

# model.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=[f1,"accuracy"])

# print('running 2 epochs with non-trainable base layers ...\n')
# history = model.fit_generator(train_gen,
#                               steps_per_epoch = np.ceil(len(train_ids)/bs),
#                               epochs = 2,
#                               validation_data = val_gen,
#                               validation_steps = np.ceil(len(val_ids)/bs),
#                               use_multiprocessing = True,
#                               callbacks = cbks1)



print('unfreezing all layers ...\n')
for layer in model.layers:
    layer.trainable = True


# For Differential Learning Rate
split_layer_1 = [layer for layer in model.layers if layer.name == model_split_1][0]
split_layer_2 = [layer for layer in model.layers if layer.name == model_split_2][0]

opt = Adam_dlr(split_l1 = split_layer_1,
              split_l2 = split_layer_2,
              lr = [1e-10, 1e-7, 5*1e-3])

model.compile(loss='mse',
              optimizer=opt,
              metrics=["accuracy"])

history = model.fit_generator(train_gen,
                              steps_per_epoch = np.ceil(len(train_ids)/bs),
                              epochs = nb_epoch,
                              validation_data = val_gen,
                              validation_steps = np.ceil(len(val_ids)/bs),
                              use_multiprocessing = True,
                              callbacks = cbks)


/usr/local/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


unfreezing all layers ...

Epoch 1/2
7/7 [==============================] - 97s 14s/step - loss: 2258.7995 - acc: 0.0045 - val_loss: 9378.0561 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 9378.05607, saving model to ./models/iterations/03222019_resnet50_regression/resnet50_regression_checkpoint.h5
Epoch 2/2
7/7 [==============================] - 65s 9s/step - loss: 1326.5105 - acc: 0.0029 - val_loss: 5495.2629 - val_acc: 0.0000e+00

Epoch 00002: val_loss improved from 9378.05607 to 5495.26293, saving model to ./models/iterations/03222019_resnet50_regression/resnet50_regression_checkpoint.h5


In [23]:
## Save model and weights
model_json = model.to_json()
with open(save_model_path + model_name + '_model.json', "w", encoding = 'utf-8') as json_file:
    json_file.write(model_json)
model.save_weights(save_model_path + model_name + '_weights.h5')

#Calculate execution time
end = time.time()
dur = end-start

if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours")

Execution Time: 3.9354347467422484 minutes


In [24]:
## Reading the true annotations
data = pd.read_csv(annotation_path, encoding = 'latin-1')
columns = 'Col8'
data = data.loc[~pd.isnull(data['Col8']),:]

image_names = [i for i in os.listdir(test_path) if 'Thumbs' not in i]

data['filename'] = data['filename'] + '.jpg'
test_df = data.loc[data['filename'].isin(image_names)].drop_duplicates().reset_index(drop = True)
print(test_df.shape[0], 'test images found ...')

model_path = save_model_path

## Load model and weights
print('loading the model ...\n')
json_file = open(model_path + 'resnet50_regression_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(model_path + 'resnet50_regression_checkpoint.h5')

70 test images found ...
loading the model ...



In [25]:
## Defining augmentation techniques for Test Time Augmentation
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.OneOf([
            sometimes(iaa.CropAndPad(percent=(-0.05, 0.1))), # zoom in
            sometimes(iaa.Affine(scale={"x": (0.6, 1.2), "y": (0.6, 1.2)})) # zoom out
        ]), 
        # execute 1 to 2 of the following augmenters per image
        sometimes(iaa.Affine(translate_px={"x": (0, 10), "y": (0, 10)},
                            rotate = (-25, 25)
                            )
                 ), # horizontal/vertical shift
        iaa.SomeOf((1, 4),
                   [iaa.OneOf([
                       iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)),
                       iaa.GaussianBlur(sigma=(0, 0.7))
                   ]),
                    sometimes(iaa.Dropout((0.01, 0.02), per_channel=0.5)),
                    iaa.AdditiveGaussianNoise(loc=32, scale=0.01*255), # white noise
                    iaa.Add((-20, 50)), # brightness
                    iaa.OneOf([
                        iaa.LinearContrast(alpha=(0.5,1.2), per_channel = True),
                        iaa.ContrastNormalization((0.5, 1.0))
                    ])
            ], random_order=True)
    ], random_order=True)

In [47]:
## Get actual code
y_true = np.array(test_df[columns])
y_true.shape

(70,)

In [48]:
## Get predicted code
pred = []
for ind in range(test_df.shape[0]):

    image_n = test_df.loc[ind, 'filename']
    ## Read Image
    img = cv2.cvtColor(cv2.imread(test_path + '/' + image_n), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_rows, img_cols))

    images_l = []
    ## Original Image
    images_l.append(img/255)
    ## Augmented Images
    for aug in range(1):
        images_l.append(seq.augment_image(img)/255)

    img_predictions = []
    for image in images_l:
        img_predictions.append(model.predict(image.reshape((-1,img_rows, img_cols, 3)))[0,0])
    pred.append(np.round(np.mean(img_predictions),0))
    
y_pred = np.array(pred)

In [55]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## Accuracy
print('MAPE: ',np.round(mean_absolute_percentage_error(y_true, y_pred),3))

Accuracy:  0.014
MAPE:  66.893
